In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Aditya\\Documents\\Aditya\\Hackathons\\init_prep\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Aditya\\Documents\\Aditya\\Hackathons\\init_prep'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_test_size: float
    params_random_state: int
    params_probability: bool

In [6]:
from heartDiseaseClassification.constants import *
from heartDiseaseClassification.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_test_size=self.params.TEST_SIZE,
            params_random_state=self.params.RANDOM_STATE,
            params_probability=self.params.PROBABILITY
        )
        return prepare_base_model_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
import joblib

C:\Users\Aditya\AppData\Local\Temp\ipykernel_18328\1018924555.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


[2024-02-06 02:30:09,113: WARNING: module_wrapper: From c:\Users\Aditya\Documents\Aditya\Hackathons\init_prep\venv\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [9]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        logistic_regression = LogisticRegression(random_state=self.config.params_random_state)
        random_forest = RandomForestClassifier(random_state=self.config.params_random_state)
        decision_tree = DecisionTreeClassifier(random_state=self.config.params_random_state)
        svc = SVC(probability=self.config.params_probability, random_state=self.config.params_random_state)

        self.model = VotingClassifier(estimators=[('lr', logistic_regression), ('rf', random_forest), ('dt', decision_tree), ('svc', svc)], voting='soft')

        self.save_model(path=self.config.base_model_path, model=self.model)
    
    @staticmethod
    def _prepare_full_model(model, X_train_scaled, y_train):
        full_model = model.fit(X_train_scaled, y_train)
        # y_pred = full_model.predict(X_train_scaled)
        # print("Classification Report:\n", classification_report(y_train, y_pred))
        return full_model
    
    def update_base_model(self):
        filePath = f"{self.config.root_dir.parent}/data_ingestion/my_data.csv"
        data = pd.read_csv(filePath)
        imputer = SimpleImputer(strategy='mean')
        df_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
        X = df_imputed.drop('target', axis=1)
        y = df_imputed['target']
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)
        y_test = pd.DataFrame(y_test, columns=['target'])
        y_train = pd.DataFrame(y_train, columns=['target'])
        X_train_scaled.to_csv(
            f"{self.config.root_dir.parent}/data_ingestion/X_train_scaled.csv", index=False)
        X_test_scaled.to_csv(
            f"{self.config.root_dir.parent}/data_ingestion/X_test_scaled.csv", index=False)
        y_train.to_csv(
            f"{self.config.root_dir.parent}/data_ingestion/y_train.csv", index=False)
        y_test.to_csv(
            f"{self.config.root_dir.parent}/data_ingestion/y_test.csv", index=False)
        self.full_model = self._prepare_full_model(
            model = self.model,
            X_train_scaled = X_train_scaled,
            y_train = y_train
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        try:
            joblib.dump(model, path)
        except Exception as e:
            raise Exception(f"Error in saving model: {e}")

In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-02-06 02:30:09,438: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-06 02:30:09,441: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-06 02:30:09,443: INFO: common: created directory at: artifacts]
[2024-02-06 02:30:09,444: INFO: common: created directory at: artifacts/prepare_base_model]


c:\Users\Aditya\Documents\Aditya\Hackathons\init_prep\venv\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Aditya\Documents\Aditya\Hackathons\init_prep\venv\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
